In [2]:
from importlib import reload
import numpy as np
import folium
import src.localization as localization
reload(localization)
from src.localization import read_data
from src.localization import plot_coeffs
from src.localization import get_distance_in_meters
from src.localization import gen_regressors
from src.localization import gen_fingerprints
from src.localization import cell_search
import pandas as pd

In [12]:
positions, df_points = read_data('LocTreino_Equipe_7.csv')

In [16]:
bts_positions, df_btss = read_data('Bts.csv')

In [25]:
center_lat = np.mean([lat for lat in bts_positions[:, 0]])
center_lon = np.mean([lon for lon in bts_positions[:, 1]])
original_pts_map = folium.Map(location=[center_lat, center_lon],
                              zoom_start=15,
                              tiles='CartoDB positron')

In [40]:
positions

array([[ -8.0535 , -34.94862],
       [ -8.05004, -34.95311],
       [ -8.04802, -34.94762],
       ...,
       [ -8.05233, -34.94834],
       [ -8.05559, -34.95374],
       [ -8.0537 , -34.95964]])

In [27]:
# Plot points
for location in positions:
    marker = folium.CircleMarker(location=location,
                                 radius=0.5,
                                 color='red',
                                 fill_opacity=0.1)
    marker.add_to(original_pts_map)

# Plot BTSs locations
for location in bts_positions:
    marker = folium.Marker(location=location)
    marker.add_to(original_pts_map)

# Export map
original_pts_map.save("original_pts.html")

In [28]:
from sklearn import neighbors

In [29]:
# Choose columns to keep in X 
# (remove unimportant/target columns)
columns = [c for c in df_points.columns if c not in ['pontoId',
                                                     'lat',
                                                     'lon']]

# X
samples = df_points[columns].values

# Y_lat
target_lat = df_points['lat'].values
# Y_lon
target_lon = df_points['lon'].values

In [30]:
lat_reg = neighbors.KNeighborsRegressor(n_neighbors=1)
lon_reg = neighbors.KNeighborsRegressor(n_neighbors=1)

In [31]:
lat_reg.fit(samples, target_lat)
lon_reg.fit(samples, target_lon)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                    weights='uniform')

In [32]:
predicted_lat = lat_reg.predict(samples)
predicted_lon = lon_reg.predict(samples)

In [35]:
# from folium.plugins import MarkerCluster
folium_map = folium.Map(location=[center_lat, center_lon],
                        zoom_start=15,
                        tiles='CartoDB positron')
# cluster = MarkerCluster(list(zip(predicted_lat, predicted_lon))).add_to(folium_map)
predicted_positions = list(zip(predicted_lat, predicted_lon))
for location in predicted_positions:
    marker = folium.CircleMarker(location=location,
                                 color='red',
                                 weight=1,
                                 radius=3)
    marker.add_to(folium_map)
for location in bts_positions:
    marker = folium.Marker(location=location)
    marker.add_to(folium_map)
for p1, p2 in list(zip(positions, predicted_positions)):
    folium.PolyLine([p1, p2],
                    color="green",
                    weight=1,
                    opacity=0.5).add_to(folium_map)
folium_map.save("knn_result_map.html")

In [38]:
result_map = folium.Map(location=[center_lat, center_lon],
                        zoom_start=15,
                        tiles='CartoDB positron')

predicted_positions = list(zip(predicted_lat, predicted_lon))
for location in predicted_positions:
    marker = folium.CircleMarker(location=location, color='red', radius=2)
    marker.add_to(result_map)
for location in positions:
    marker = folium.CircleMarker(location=location, color='blue', radius=1)
    marker.add_to(result_map)
for location in bts_positions:
    marker = folium.Marker(location=location)
    marker.add_to(result_map)
for p1, p2 in list(zip(positions, predicted_positions)):
    folium.PolyLine([p1, p2], color="green", weight=1, opacity=0.5).add_to(result_map)
result_map.save("result_map.html")